# Implementação de Métodos de Machine Learning
**Aluno:** Matheus Gama dos Santos - 20180163117<br>
**Curso:** Ciência da Computação - UFPB<br>
**Profª:** Thais Gaudencio do Rego<br><br>
Este projeto consiste na implementação de métodos de Machine Learning para analisar as bases de dados listadas abaixo:
- [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data)
- [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)
- [2018-2019 Premier League Data
](https://www.kaggle.com/thesiff/premierleague1819)

Cada base de dados será tratada em uma das 3 seções: **Classificação**, **Regressão** e **Clusterização** (nessa ordem).<br>

## Classificação

In [1]:
#import core modules
import pandas as pd
import numpy as np
from sklearn import *

In [2]:
#load dataframe
titanic_filename = 'titanic/train.csv'
titanic_dataframe = pd.read_csv(titanic_filename)

In [3]:
titanic_dataframe.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Preprocessing

In [4]:
# select useful attributes
titanic_dataframe = titanic_dataframe.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Fare"], axis=1)
titanic_dataframe.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [5]:
# converting "Sex" atribute
titanic_dataframe.Sex = titanic_dataframe.Sex.replace({'male':0, 'female':1})
titanic_dataframe.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0


In [6]:
titanic_dataframe.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000


Analisando o estado atual do Dataframe, observa-se que o atributo "Age" possui mais de 100 linhas sem dado, com 75% dos valores igual ou abaixo de 38.